In [16]:
%reload_ext autoreload
%autoreload 2
from aldiscore.prediction.extractor import FeatureExtractor
from aldiscore.prediction.predictor import DifficultyPredictor
from aldiscore.prediction import utils
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
import os
from pathlib import Path
from Bio import SeqIO
from aldiscore import ROOT, get_from_config
import numpy as np
import pandas as pd

Trials:
- tc_base vs tc_different_statistics!
- tc_base vs tc_dist_abs vs tc_dist_scaled vs tc_dist_log
- tc_base + (tc_dist_abs vs tc_dist_scaled vs tc_dist_log)
- tc_base vs tc_50 vs tc_100

TODOs:
- Try new/better aggregation statistics
- Add remaining PSA features
- Look into simulation paper again


In [ ]:
print(np.std([1, 2, 3]))
print(np.std([1, 1, 2, 2, 3, 3]))

0.816496580927726
0.816496580927726


In [ ]:
TEST_FILE_AA = Path("../data/aa.fasta")
TEST_FILE_DNA = Path("data/dna.fasta")
INTRON_FILE = Path("../../../data/size_test/s_10.fasta")

sequences = list(SeqIO.parse(INTRON_FILE, "fasta"))
for seq_record in sequences:
    seq_record.seq = Seq(str(seq_record.seq).replace("-", ""))

print("k =", len(sequences))

seq_lens = [len(seq) for seq in sequences]
print(max(seq_lens))

# print(sequences[0].seq[:100])
# print(sequences[29].seq[:100])
psa_config = {"MAX_PSA_COUNT": 100}
extractor = FeatureExtractor(sequences, psa_config=psa_config, track_perf=True)
feats = extractor.compute()
print(extractor._perf_dict)

# print(DifficultyPredictor().predict(INTRON_FILE, drop_gaps=True))
# feats.loc[:, feats.columns.str.contains("chi-square")]
# feats.loc[:, feats.columns.str.contains("tc_")]

k = 10
3786
{'_init_basics': 0.0026233457028865814, '_init_char_dists': 0.0023433156311511993, '_init_psa': 4.891748297959566, '_init_psa_index_map': 0.002786535769701004, '_data_type': 2.466142177581787e-06, '_num_sequences': 2.4512410163879395e-06, '_sequence_length': 0.00028173625469207764, '_sequence_length_ratio': 1.2252479791641235e-05, '_lower_bound_gap_percentage': 1.7035752534866333e-05, '_sequence_entropy': 0.00017172470688819885, '_char_js_divergence': 0.0002912767231464386, '_homopolymer_js_divergence': 0.01988953724503517, '_transitive_consistency': 0.009541474282741547, '_psa_basic_features': 0.001590825617313385, '_psa_gap_length': 0.012303106486797333, '_kmer_similarity': 0.142743282020092}
0.05669790811394097


In [5]:
perf_dir = ROOT / "logs" / "perf"
perf_file = sorted(os.listdir(perf_dir))[-1]
print(perf_file)
perf_df = pd.read_parquet(perf_dir / perf_file)
perf_df
# perf_df.sum(axis=1).groupby("source").median()

2025-10-02_13-43-37.parquet


_init_basics  _init_char_dists  \
source       dataset                                                         
bralibase_k5 HIV_FE.apsi-86.sci-94.no-5         0.113276          0.000500   
             HCV_SLIV.apsi-86.sci-100.no-1      0.081990          0.000302   
             HIV_FE.apsi-92.sci-94.no-1         0.088590          0.000486   
             HIV_FE.apsi-87.sci-90.no-2         0.092224          0.000393   
             HIV_GSL3.apsi-88.sci-77.no-2       0.060637          0.000392   
...                                                  ...               ...   
treebase_v1  20196_21.phy                       0.004679          0.002546   
             14688_30.phy                       0.007059          0.000350   
             18202_8.phy                        0.017448          0.029228   
             16461_8.phy                        0.016802          0.018810   
             11746_2.phy                        0.008457          0.013142   

                                            _init_psa  _init_psa_index_map  \
source       dataset                                                         
bralibase_k5 HIV_FE.apsi-86.sci-94.no-5      0.013520             0.000534   
             HCV_SLIV.apsi-86.sci-100.no-1   0.013632             0.000607   
             HIV_FE.apsi-92.sci-94.no-1      0.013638             0.000589   
             HIV_FE.apsi-87.sci-90.no-2      0.013503             0.000537   
             HIV_GSL3.apsi-88.sci-77.no-2    0.014303             0.000579   
...                                               ...                  ...   
treebase_v1  20196_21.phy                    1.147992             0.031682   
             14688_30.phy                    0.021165             0.000889   
             18202_8.phy                    52.351813             0.069025   
             16461_8.phy                    26.813274             0.067993   
             11746_2.phy                    38.343293             0.055424   

                                            _data_type  _num_sequences  \
source       dataset                                                     
bralibase_k5 HIV_FE.apsi-86.sci-94.no-5       0.000009        0.000007   
             HCV_SLIV.apsi-86.sci-100.no-1    0.000010        0.000009   
             HIV_FE.apsi-92.sci-94.no-1       0.000011        0.000009   
             HIV_FE.apsi-87.sci-90.no-2       0.000013        0.000008   
             HIV_GSL3.apsi-88.sci-77.no-2     0.000010        0.000008   
...                                                ...             ...   
treebase_v1  20196_21.phy                     0.000006        0.000003   
             14688_30.phy                     0.000005        0.000003   
             18202_8.phy                      0.000006        0.000003   
             16461_8.phy                      0.000006        0.000003   
             11746_2.phy                      0.000006        0.000003   

                                            _sequence_length  \
source       dataset                                           
bralibase_k5 HIV_FE.apsi-86.sci-94.no-5             0.003386   
             HCV_SLIV.apsi-86.sci-100.no-1          0.003383   
             HIV_FE.apsi-92.sci-94.no-1             0.003171   
             HIV_FE.apsi-87.sci-90.no-2             0.003521   
             HIV_GSL3.apsi-88.sci-77.no-2           0.002693   
...                                                      ...   
treebase_v1  20196_21.phy                           0.000522   
             14688_30.phy                           0.000508   
             18202_8.phy                            0.000512   
             16461_8.phy                            0.000474   
             11746_2.phy                            0.000505   

                                            _sequence_length_ratio  \
source       dataset                                                 
bralibase_k5 HIV_FE.apsi-86.sci-94.no-5                   0.000018   
        